### GITHUB setup instructions are in a separate notebook

#### it's published on the github repo

# K-Neighbour MODEL FOR NBA ROOKIE PLAYERS

# Load Packages

In [1]:
# Load the packages needed for Modelling upfront
import pandas as pd
import numpy as np

# packages for data processing
from sklearn.preprocessing   import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

# packages for modelling
from sklearn.linear_model      import LogisticRegression
from sklearn.neighbors         import KNeighborsClassifier
from sklearn.tree              import DecisionTreeClassifier
from sklearn.ensemble          import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.svm               import SVC

# packages for evaluation metrics
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import roc_auc_score

from joblib import dump

# Load the data

In [17]:
# Load the training data set
train = pd.read_csv('/Users/jasle1/Desktop/MDSI/ADSI/AT_1/adsi_at1/data/raw/train.csv')

In [18]:
# Load the test data set
test = pd.read_csv('/Users/jasle1/Desktop/MDSI/ADSI/AT_1/adsi_at1/data/raw/test.csv')

# Explore the Data

In [153]:
# check rows and columns of the training set
train.shape

(8000, 21)

In [154]:
# check rows and columns of the test set
test.shape

(3799, 20)

In [155]:
train.head()

,Id,GP,MIN,PTS,FGM,FGA,FG%,3P Made,3PA,3P%,...,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,TARGET_5Yrs
0,10556,80,24.3,7.8,3.0,6.4,45.7,0.1,0.3,22.6,...,2.9,72.1,2.2,2.0,3.8,3.2,1.1,0.2,1.6,1
1,5342,75,21.8,10.5,4.2,7.9,55.1,-0.3,-1.0,34.9,...,3.6,67.8,3.6,3.7,6.6,0.7,0.5,0.6,1.4,1
2,5716,85,19.1,4.5,1.9,4.5,42.8,0.4,1.2,34.3,...,0.6,75.7,0.6,1.8,2.4,0.8,0.4,0.2,0.6,1
3,13790,63,19.1,8.2,3.5,6.7,52.5,0.3,0.8,23.7,...,1.5,66.9,0.8,2.0,3.0,1.8,0.4,0.1,1.9,1
4,5470,63,17.8,3.7,1.7,3.4,50.8,0.5,1.4,13.7,...,0.5,54.0,2.4,2.7,4.9,0.4,0.4,0.6,0.7,1


In [156]:
test.head()

,Id,GP,MIN,PTS,FGM,FGA,FG%,3P Made,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV
0,1,56,9.1,4.0,1.6,3.7,43.7,0.1,0.3,7.3,0.7,1.2,63.4,1.2,0.8,1.7,0.4,0.2,0.3,0.8
1,8194,43,19.3,10.1,3.7,8.1,46.0,0.6,1.7,35.1,1.8,2.5,75.3,0.5,0.9,1.5,3.5,0.6,-0.0,1.8
2,3,82,33.9,11.3,4.9,10.6,45.6,0.5,1.9,44.8,1.8,2.7,71.2,1.3,3.3,4.5,2.5,1.3,0.3,2.0
3,8196,86,44.7,18.8,6.8,15.9,42.9,0.5,1.8,13.5,4.5,6.3,70.9,1.5,3.2,5.0,4.1,0.9,0.1,3.6
4,8197,58,12.3,4.7,1.6,4.0,40.0,0.5,1.7,38.7,1.1,1.3,76.9,0.2,0.6,0.9,1.5,0.5,-0.4,0.9


In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 21 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Id           8000 non-null   int64  
 1   GP           8000 non-null   int64  
 2   MIN          8000 non-null   float64
 3   PTS          8000 non-null   float64
 4   FGM          8000 non-null   float64
 5   FGA          8000 non-null   float64
 6   FG%          8000 non-null   float64
 7   3P Made      8000 non-null   float64
 8   3PA          8000 non-null   float64
 9   3P%          8000 non-null   float64
 10  FTM          8000 non-null   float64
 11  FTA          8000 non-null   float64
 12  FT%          8000 non-null   float64
 13  OREB         8000 non-null   float64
 14  DREB         8000 non-null   float64
 15  REB          8000 non-null   float64
 16  AST          8000 non-null   float64
 17  STL          8000 non-null   float64
 18  BLK          8000 non-null   float64
 19  TOV   

In [9]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3799 entries, 0 to 3798
Data columns (total 20 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Id       3799 non-null   int64  
 1   GP       3799 non-null   int64  
 2   MIN      3799 non-null   float64
 3   PTS      3799 non-null   float64
 4   FGM      3799 non-null   float64
 5   FGA      3799 non-null   float64
 6   FG%      3799 non-null   float64
 7   3P Made  3799 non-null   float64
 8   3PA      3799 non-null   float64
 9   3P%      3799 non-null   float64
 10  FTM      3799 non-null   float64
 11  FTA      3799 non-null   float64
 12  FT%      3799 non-null   float64
 13  OREB     3799 non-null   float64
 14  DREB     3799 non-null   float64
 15  REB      3799 non-null   float64
 16  AST      3799 non-null   float64
 17  STL      3799 non-null   float64
 18  BLK      3799 non-null   float64
 19  TOV      3799 non-null   float64
dtypes: float64(18), int64(2)
memory usage: 593.7 KB


In [10]:
train.describe()

,Id,GP,MIN,PTS,FGM,FGA,FG%,3P Made,3PA,3P%,...,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,TARGET_5Yrs
count,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,...,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000
mean,6856.971000,62.777875,18.576662,7.267088,2.807037,6.231212,44.608900,0.264525,0.816562,19.583700,...,1.947788,71.365825,1.077838,2.168500,3.245300,1.624513,0.648687,0.245212,1.257763,0.833625
std,3977.447579,17.118774,8.935263,4.318732,1.693373,3.584559,6.155453,0.384093,1.060964,16.003155,...,1.252352,10.430447,0.785670,1.392224,2.085154,1.355986,0.407626,0.821037,0.723270,0.372440
min,4.000000,-8.000000,2.900000,0.800000,0.300000,0.800000,21.300000,-1.100000,-3.100000,-38.500000,...,0.000000,-13.300000,0.000000,0.200000,0.300000,0.000000,0.000000,-17.900000,0.100000,0.000000
25%,3413.750000,51.000000,12.000000,4.100000,1.600000,3.600000,40.400000,0.000000,0.100000,8.400000,...,1.000000,65.000000,0.500000,1.100000,1.700000,0.700000,0.300000,0.100000,0.700000,1.000000
50%,6787.500000,63.000000,16.800000,6.300000,2.400000,5.400000,44.400000,0.300000,0.800000,19.500000,...,1.700000,71.400000,0.900000,1.900000,2.800000,1.300000,0.600000,0.200000,1.100000,1.000000
75%,10299.250000,74.000000,23.500000,9.500000,3.700000,8.100000,48.700000,0.500000,1.500000,30.600000,...,2.600000,77.500000,1.500000,2.900000,4.300000,2.200000,0.900000,0.400000,1.600000,1.000000
max,13798.000000,123.000000,73.800000,34.200000,13.100000,28.900000,67.200000,1.700000,4.700000,82.100000,...,11.100000,168.900000,5.500000,11.000000,15.900000,12.800000,3.600000,18.900000,5.300000,1.000000


# Transform the Data

In [19]:
# extract the target variable out as y = target
y_train = train.pop('TARGET_5Yrs')
y_train.shape

(8000,)

In [20]:
# create a copy of test set for use later - this will merge with the prediction file for kaggle submission
test_ID = test.copy()
test_ID

,Id,GP,MIN,PTS,FGM,FGA,FG%,3P Made,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV
0,1,56,9.1,4.0,1.6,3.7,43.7,0.1,0.3,7.3,0.7,1.2,63.4,1.2,0.8,1.7,0.4,0.2,0.3,0.8
1,8194,43,19.3,10.1,3.7,8.1,46.0,0.6,1.7,35.1,1.8,2.5,75.3,0.5,0.9,1.5,3.5,0.6,-0.0,1.8
2,3,82,33.9,11.3,4.9,10.6,45.6,0.5,1.9,44.8,1.8,2.7,71.2,1.3,3.3,4.5,2.5,1.3,0.3,2.0
3,8196,86,44.7,18.8,6.8,15.9,42.9,0.5,1.8,13.5,4.5,6.3,70.9,1.5,3.2,5.0,4.1,0.9,0.1,3.6
4,8197,58,12.3,4.7,1.6,4.0,40.0,0.5,1.7,38.7,1.1,1.3,76.9,0.2,0.6,0.9,1.5,0.5,-0.4,0.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3794,8175,84,21.2,8.7,3.4,6.7,50.2,0.0,-0.0,-4.5,1.7,2.5,68.1,1.9,2.3,3.9,1.5,0.6,0.3,2.0
3795,8176,49,16.3,6.4,2.9,6.6,44.4,-0.1,-0.4,19.8,1.0,1.9,50.2,1.7,2.8,4.4,0.4,0.4,0.4,0.7
3796,8178,53,9.9,2.1,0.8,1.8,43.1,-0.4,-0.6,1.9,0.6,1.0,63.9,0.7,1.0,1.7,0.4,0.4,0.2,0.5
3797,8181,89,38.3,14.5,5.4,11.8,45.2,0.5,1.2,24.7,2.5,2.9,89.2,1.5,4.0,5.5,3.7,1.3,0.3,2.4


In [21]:
# Standardise the dataset
scaler = StandardScaler()
train = scaler.fit_transform(train)
test = scaler.fit_transform(test)

In [22]:
# Split the training data into train and validation sets for evaluation
# set the validation set at 20% of the full training set
X_train, X_val, y_train, y_val = train_test_split(train, y_train, test_size=0.2, random_state = 42)

# Run Grid-Search

In [23]:
# Knearest neighbour

knn = KNeighborsClassifier()

knn_params = {'n_neighbors': [5, 10, 15],
              'weights': ['uniform', 'distance']
}

model = GridSearchCV(knn, param_grid=knn_params, cv=3, verbose = 1)

# Train K-Nearest Neighbors Model

In [24]:
model.fit(X_train, y_train)

print(f'CrossVal Score: {model.best_score_}')
print(f'Training Score: {model.score(X_train, y_train)}')
print(f'Testing Score: {model.score(X_val, y_val)}')
print(model.best_params_)

Fitting 3 folds for each of 6 candidates, totalling 18 fits
CrossVal Score: 0.8303124184850228
Training Score: 1.0
Testing Score: 0.829375
{'n_neighbors': 15, 'weights': 'distance'}


In [25]:
# Make predictions on the validation dataset
val_pred = model.predict_proba(X_val)
val_pred

array([[0.20371138, 0.79628862],
       [0.22142788, 0.77857212],
       [0.26250275, 0.73749725],
       ...,
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.12917309, 0.87082691]])

# Evaluate Model Performance

In [26]:
# Calculate auc scores for performance evaluation
auc_score = roc_auc_score(y_val, val_pred[:,1])

print(f'The Logistic regression ROC AUC score is {auc_score}')

The Logistic regression ROC AUC score is 0.6486032232652981


In [27]:
# Make predictions on the test set now
test_pred = model.predict_proba(test)
test_pred

array([[0.29188499, 0.70811501],
       [0.12920078, 0.87079922],
       [0.        , 1.        ],
       ...,
       [0.15272233, 0.84727767],
       [0.06526444, 0.93473556],
       [0.12380755, 0.87619245]])

In [28]:
# *** save scaler in the models folder called "scaler.joblib" for use next time
dump(model, '/Users/jasle1/Desktop/MDSI/ADSI/AT_1/adsi_at1/models/KNN_scaled_2.11.joblib')

['/Users/jasle1/Desktop/MDSI/ADSI/AT_1/adsi_at1/models/KNN_scaled_2.11.joblib']

# Prepare the file for extracting final prediction output - for Kaggle submission

In [29]:
# Convert the datasets into pandas dataframe for easy merge with prediction file
test_id = pd.DataFrame(test_ID.iloc[:,0])
test_id

,Id
0,1
1,8194
2,3
3,8196
4,8197
...,...
3794,8175
3795,8176
3796,8178
3797,8181


In [30]:
# Convert the datasets into pandas dataframe for easy merge with prediction file
test_pred = pd.DataFrame(test_pred[:,1])
test_pred.head()
# it seems that the framework is giving out prediction for both yes and no

,0
0,0.708115
1,0.870799
2,1.000000
3,1.000000
4,0.804196


In [12]:
# Now, merge the test data set with predictions data
df = pd.concat([test, test_pred], axis=1)
df.head()

,Id,GP,MIN,PTS,FGM,FGA,FG%,3P Made,3PA,3P%,...,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,0
0,1,56,9.1,4.0,1.6,3.7,43.7,0.1,0.3,7.3,...,1.2,63.4,1.2,0.8,1.7,0.4,0.2,0.3,0.8,0.800000
1,8194,43,19.3,10.1,3.7,8.1,46.0,0.6,1.7,35.1,...,2.5,75.3,0.5,0.9,1.5,3.5,0.6,-0.0,1.8,0.800000
2,3,82,33.9,11.3,4.9,10.6,45.6,0.5,1.9,44.8,...,2.7,71.2,1.3,3.3,4.5,2.5,1.3,0.3,2.0,0.800000
3,8196,86,44.7,18.8,6.8,15.9,42.9,0.5,1.8,13.5,...,6.3,70.9,1.5,3.2,5.0,4.1,0.9,0.1,3.6,0.733333
4,8197,58,12.3,4.7,1.6,4.0,40.0,0.5,1.7,38.7,...,1.3,76.9,0.2,0.6,0.9,1.5,0.5,-0.4,0.9,0.800000


In [31]:
# Now, merge the test data set with predictions data - IF SCALED
df = pd.concat([test_id, test_pred], axis=1)
df.head()

,Id,0
0,1,0.708115
1,8194,0.870799
2,3,1.000000
3,8196,1.000000
4,8197,0.804196


In [32]:
# Rename the '0' prediction column to align with Kaggle submission requirements
df_final = df.rename(columns={0: 'TARGET_5Yrs'})
df_final.head()

,Id,TARGET_5Yrs
0,1,0.708115
1,8194,0.870799
2,3,1.000000
3,8196,1.000000
4,8197,0.804196


In [33]:
# Extract the ID and Prediction columns for Kaggle Submission
submission = df_final[['Id', 'TARGET_5Yrs']]
submission.head()

,Id,TARGET_5Yrs
0,1,0.708115
1,8194,0.870799
2,3,1.000000
3,8196,1.000000
4,8197,0.804196


In [34]:
# save the submission file for kaggle upload
submission.to_csv('jasleen_KNN_scaled_week2.11', index=False)